In [1]:
import os
from maze_env import Maze
from utility import RL
import time

In [2]:
# def onKeyPress(event):
#     is_terminate = False
#     if (event.char == "w"):
#         is_terminate, reward = env.take_action(0, show_animate=True)
#     elif (event.char == "d"):
#         is_terminate, reward = env.take_action(1, show_animate=True)
#     elif (event.char == "s"):
#         is_terminate, reward = env.take_action(2, show_animate=True)
#     elif (event.char == "a"):
#         is_terminate, reward = env.take_action(3, show_animate=True)
#     elif (event.char == "q"):
#         env.destroy()
#     if (is_terminate):
#         env.reset()

In [3]:
# if __name__ == "__main__":
env = Maze()
rl = RL(env)
print('{0}x{1}, act_space = {2}'.format(env.MAZE_Limit[0],env.MAZE_Limit[1], len(env.action_space)))
# env.bind('<KeyPress>', onKeyPress)
# env.mainloop()

4x4, act_space = 4


# DP Prediction

In [4]:
rl.DP.iteration(update = 10)

9 [[ 0. -1. -2. -3.]
 [-1. -2. -3. -2.]
 [-2. -3. -2. -1.]
 [-3. -2. -1.  0.]]


# model-free MC

In [5]:
rl.MF.iteration(n_episode = 3000,  model = "MC")

The model is MC
Value
 [[ 0.         -5.32745919 -7.20154658 -7.66815129]
 [-5.33253063 -6.56402402 -7.16914477 -7.10629757]
 [-7.09470096 -7.15290231 -6.49034301 -5.13943962]
 [-7.62512622 -7.08552494 -5.19520732  0.        ]]
--------------------------



# model-free TD(0)

In [6]:
rl.MF.iteration(n_episode = 3000,  model = "TD_0")

The model is TD_0
Value
 [[ 0.         -4.63435594 -6.67989484 -8.05624387]
 [-2.94009187 -5.09539287 -6.94538239 -6.17609954]
 [-8.53239277 -7.99638666 -7.15658759 -5.64660028]
 [-7.93577635 -5.56510776 -2.28809012  0.        ]]
--------------------------



# model-free TD(n)

In [7]:
rl.MF.iteration(n_episode = 3000,  model = "TD_n")

The model is TD_n
Value
 [[ 0.         -3.32354864 -4.75412479 -5.60188957]
 [-5.0722291  -6.46102069 -4.69605985 -3.25464941]
 [-5.76468539 -6.33780824 -4.02201928 -1.88887814]
 [-6.3106283  -6.59035318 -5.50017062  0.        ]]
--------------------------



# model-free TD(lambda)

In [8]:
rl.MF.iteration(n_episode = 3000,  model = "TD_lambda")

The model is TD_lambda
Value
 [[ 0.         -5.96697182 -5.48859729 -5.54267772]
 [-4.07498411 -5.18287474 -6.00498823 -4.29797792]
 [-4.93755872 -4.37231352 -3.70538212 -1.46307163]
 [-4.04369982 -4.32629094 -2.67918492  0.        ]]
--------------------------



# testing field

In [9]:
import numpy as np
a = np.array([1,2,3,4,5])

In [10]:
np.logspace(start=1,stop=len(a),num=len(a),endpoint=True,base=2)/2

array([ 1.,  2.,  4.,  8., 16.])